In [ ]:
from hytek_parser import hy3_parser

In [7]:
file_path = "../SampleMeetResults/Meet Results-PCS Spring Home Meet 2025-22Mar2025-002.hy3"

parsed_file = hy3_parser.parse_hy3(file_path)
print(parsed_file)

Invalid line code: B3


RuntimeError: Exception while parsing, please open an issue with full traceback at https://github.com/SwimComm/hytek-parser/issues/new/choose